# Real-time Implementation and Automation with FXCM 

---------------------------------------------

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with an FXCM Practice/Demo Account and NOT with a Live Trading Account._

----------------------------------------------------

## Recap: Historical Data, real-time Data and Orders

In [ ]:
import pandas as pd
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
api.get_candles("EUR/USD", start = "2020-07-01", end = "2020-07-31",  period = "D1")

In [ ]:
api.get_candles("EUR/USD", number = 10, period = "m1")

In [ ]:
api.subscribe_market_data("EUR/USD")

In [ ]:
api.get_subscribed_symbols()

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
def print_data(data, dataframe):
    print('%3d | %s | %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1]))

In [ ]:
api.subscribe_market_data("EUR/USD", (print_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
api.get_open_positions()

In [ ]:
api.create_market_buy_order("EUR/USD", 100)

In [ ]:
order = api.create_market_buy_order("EUR/USD", 100)

In [ ]:
order.get_time()

In [ ]:
order.get_isBuy()

In [ ]:
order.get_currency()

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
api.get_open_positions()[col]

In [ ]:
api.create_market_sell_order("EUR/USD", 200)

In [ ]:
api.get_open_positions()[col]

In [ ]:
api.close_all_for_symbol("EUR/USD")

In [ ]:
api.get_open_positions()

In [ ]:
api.get_closed_positions()[col]

In [ ]:
api.get_closed_positions_summary()[col]

In [ ]:
api.get_accounts().T

In [ ]:
api.close()

## Preview: A Trader Class live in action

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
import time
from datetime import datetime

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
class ConTrader():
    
    def __init__(self, instrument, bar_length, window, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0

    def report_trade(self, order, going):  
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

__Simple Contrarian: Bar_lenght = 1min | Window = 1 (1 minute)__

In [ ]:
trader = ConTrader("EUR/USD", bar_length = "1min", window = 1, units = 100)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0:
    api.close_all_for_symbol(trader.instrument)
    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
    time.sleep(20)
    print(api.get_closed_positions_summary()[col])
    trader.position = 0

__Simple Contrarian: Bar_lenght = 1min | Window = 60 (1 hour)__

In [ ]:
trader = ConTrader("EUR/USD", bar_length = "1min", window = 60, units = 100)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0:
    api.close_all_for_symbol(trader.instrument)
    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
    time.sleep(20)
    print(api.get_closed_positions_summary()[col])
    trader.position = 0

In [ ]:
api.close()

## Collecting and storing real-time tick data

In [ ]:
import pandas as pd
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
def print_data(data, dataframe):
    print(dataframe)

In [ ]:
api.subscribe_market_data("EUR/USD", (print_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
tick_data = None

In [ ]:
def get_tick_data(data, dataframe):
    
    global tick_data # global variable tick_data
    
    print('%3d | %s | %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1]))
    
    tick_data = dataframe.iloc[:, :2]

In [ ]:
api.subscribe_market_data("EUR/USD", (get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
tick_data

In [ ]:
tick_data.info()

In [ ]:
api.close()

## Storing and resampling real-time tick data (Part 1)

In [ ]:
import pandas as pd
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file = "fxcm.cfg")

In [ ]:
tick_data = None
resamp = None
bar_length = "10s"

In [ ]:
def get_tick_data(data, dataframe):
    
    global tick_data, resamp # global variables
    
    print('%3d | %s | %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1]))
    
    tick_data = dataframe.iloc[:, :2]
    resamp = tick_data.resample(bar_length, label = "right").last().ffill().iloc[:-1]
    resamp["Mid"] = (resamp.Ask + resamp.Bid)/2

In [ ]:
api.subscribe_market_data("EUR/USD", (get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
tick_data

In [ ]:
resamp

In [ ]:
api.close()

## A Contrader Class

In [ ]:
import pandas as pd
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file = "fxcm.cfg")

In [ ]:
class ConTrader():
    def __init__(self, instrument, bar_length):
        self.instrument = instrument
        self.bar_length = bar_length
        self.tick_data = None
        self.data = None
        self.ticks = 0 # running Tick number
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ") # Print running Tick number
        
        # collect and store tick data (mid-price only)
        self.tick_data = dataframe.iloc[:, :2].copy()
        self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
        self.tick_data = self.tick_data[self.instrument].to_frame()
        
        self.resample_and_join()
            
    def resample_and_join(self):
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]

In [ ]:
for i in range(10):
    print(i, end = " ")

In [ ]:
trader = ConTrader("EUR/USD", "10s")

In [ ]:
trader

In [ ]:
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
trader.data

In [ ]:
trader.tick_data

In [ ]:
api.close()

## Storing and resampling real-time tick data (Part 2)

__When should we resample?__<br>
Whenever there are more than 5s (__bar_length__) between the __most recent tick__ and __last full bar__. 

In [ ]:
import pandas as pd
import fxcmpy
from datetime import datetime

In [ ]:
api = fxcmpy.fxcmpy(config_file = "fxcm.cfg")

In [ ]:
class ConTrader():
    def __init__(self, instrument, bar_length):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) # Pandas Timedelta Object
        self.tick_data = None
        self.data = None
        self.ticks = 0
        self.last_bar = pd.to_datetime(datetime.utcnow()) # Time at instantiation
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms") # Pandas Timestamp Object
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.iloc[:, :2].copy()
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            
    def resample_and_join(self):
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
        self.last_bar = self.data.index[-1]  # update time of last full bar

In [ ]:
datetime.utcnow()

In [ ]:
pd.to_datetime(datetime.utcnow())

In [ ]:
pd.to_timedelta("10s")

In [ ]:
trader = ConTrader("EUR/USD", "10s")
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
trader.data

In [ ]:
trader.tick_data

In [ ]:
api.close()

## Storing and resampling real-time tick data (Part 3)

In [ ]:
import pandas as pd
import fxcmpy
from datetime import datetime

In [ ]:
api = fxcmpy.fxcmpy(config_file = "fxcm.cfg")

Goal: Only use/store the latests ticks since the (second) last full bar.

In [ ]:
class ConTrader():
    def __init__(self, instrument, bar_length):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.data = pd.DataFrame() # NEW
        self.ticks = 0
        self.last_bar = pd.to_datetime(datetime.utcnow()) 
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            #self.tick_data = dataframe.iloc[:, :2].copy()
            # NEW: get most recent ticks since last full bar
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            
    def resample_and_join(self):
        #self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
        # NEW: append the most recent ticks (resampled) to self.data
        self.data = self.data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.data.index[-1]  

In [ ]:
trader = ConTrader("EUR/USD", "10s")
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
trader.data

In [ ]:
trader.tick_data

In [ ]:
api.close()

## Working with historical data and real-time tick data (Part 1)

In [ ]:
import pandas as pd
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
instrument = "EUR/USD"

In [ ]:
df = api.get_candles(instrument, number = 250, period = "m1", columns = ["bidclose", "askclose"])
df

In [ ]:
df[instrument] = (df.bidclose + df.askclose) / 2

In [ ]:
df

In [ ]:
df = df[instrument].to_frame()

In [ ]:
df = df.resample("20min", label = "right").last().dropna().iloc[:-1]
df

In [ ]:
df.info()

In [ ]:
api.close()

## Working with historical data and real-time tick data (Part 2)

In [ ]:
import pandas as pd
import fxcmpy
from datetime import datetime

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
class ConTrader():
    def __init__(self, instrument, bar_length):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.data = None # first defined in get_most_recent()
        self.ticks = 0
        self.last_bar = None # first defined in get_most_recent()
    
    def get_most_recent(self, period = "m1", number = 10000): # NEW
        df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
        df[self.instrument] = (df.bidclose + df.askclose) / 2
        df = df[self.instrument].to_frame()
        df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
        self.data = df.copy() # first defined
        self.last_bar = self.data.index[-1] # first defined
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            
    def resample_and_join(self):
        self.data = self.data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.data.index[-1]  

In [ ]:
trader = ConTrader("EUR/USD", "1min")

In [ ]:
print(datetime.utcnow())
trader.get_most_recent(period = "m1", number = 10000)
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
trader.data.tail(10)

In [ ]:
api.close()

## Working with historical data and real-time tick data (Part 3)

In [ ]:
import pandas as pd
import fxcmpy
from datetime import datetime
import time

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
class ConTrader():
    def __init__(self, instrument, bar_length):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        
        self.count = 0 # for demonstration purposes only
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True: # repeat until we get all historical bars
            self.count += 1 # for demonstration purposes only
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.data = df.copy()
            self.last_bar = self.data.index[-1]
            # accept, if less than [bar_lenght] has elapsed since the last full historical bar and now
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            
    def resample_and_join(self):
        self.data = self.data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.data.index[-1]  

In [ ]:
trader = ConTrader("EUR/USD", "1min")

In [ ]:
print(datetime.utcnow())
trader.get_most_recent()
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
trader.data.tail(10)

In [ ]:
trader.tick_data

In [ ]:
trader.count

In [ ]:
api.close()

## Defining a Simple Contrarian Trading Strategy

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
from datetime import datetime
import time

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
class ConTrader():
    def __init__(self, instrument, bar_length, window):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None # NEW
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy() # raw!
            self.last_bar = self.raw_data.index[-1] # raw!
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() # Prepare Data / Strategy Features
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1]) # raw!
        self.last_bar = self.raw_data.index[-1] # raw!  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy() # self.raw_data new!
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()  # first defined here
        

In [ ]:
trader = ConTrader("EUR/USD", "1min", window = 1)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
trader.raw_data.tail(10)

In [ ]:
trader.data.tail(10)

In [ ]:
api.close()

## Placing Orders and Executing Trades

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
from datetime import datetime
import time

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

__Assumption: Netting Account and NOT Hedging Account (Practice Account: only Hedging is available)__ <br>
(Consequence for Trading Sessions with Practice Account: Double Costs)

In [ ]:
# Assumption: Netting Account and NOT Hedging Account (Practice Account: only Hedging is available)

class ConTrader():
    def __init__(self, instrument, bar_length, window, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units # NEW
        self.position = 0 # NEW
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades() # NEW!
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self): # NEW!
        if self.data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                print("GOING LONG")
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                print("GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: # if position is short -> go/stay short 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                print("GOING SHORT")
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                print("GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                print("GOING NEUTRAL")
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                print("GOING NEUTRAL")
            self.position = 0
        

In [ ]:
trader = ConTrader("EUR/USD", "1min", window = 1, units = 100)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data("EUR/USD", (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
api.get_open_positions()#[col]

In [ ]:
api.close_all_for_symbol("EUR/USD")

In [ ]:
api.get_closed_positions()[col]

In [ ]:
api.get_closed_positions_summary()[col]

In [ ]:
trader.data.tail(10)

In [ ]:
api.close()

## Trade Monitoring and Reporting

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
import time
from datetime import datetime

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
class ConTrader():
    
    def __init__(self, instrument, bar_length, window, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING LONG")  # NEW
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING LONG")  # NEW
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING SHORT")  # NEW
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING SHORT")  # NEW
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            self.position = 0

    def report_trade(self, order, going):  # NEW
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

In [ ]:
trader = ConTrader("EUR/USD", "1min", window = 1, units = 100)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0: # if we have final open position(s) (netting and hedging)
    api.close_all_for_symbol(trader.instrument)
    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
    time.sleep(20)
    print(api.get_closed_positions_summary()[col])
    trader.position = 0

In [ ]:
trader.data.tail(12)

In [ ]:
api.close()

## Trading other Strategies - Coding Challenge

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
import time
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

__Strategy 1__: SMA 50/200 (minutes) Crossover (200 ticks)

## Stop here if you don´t want to see the solution!

###############################################################

In [ ]:
class SMATrader():
    
    def __init__(self, instrument, bar_length, SMA_S, SMA_L, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        
        #*****************add strategy-specific attributes here******************
        self.SMA_S = SMA_S
        self.SMA_L = SMA_L
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["SMA_S"] = df[self.instrument].rolling(self.SMA_S).mean()
        df["SMA_L"] = df[self.instrument].rolling(self.SMA_L).mean()
        df["position"] = np.where(df["SMA_S"] > df["SMA_L"], 1, -1)
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING LONG") 
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING LONG") 
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0

    def report_trade(self, order, going):
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

In [ ]:
trader = SMATrader("EUR/USD", "1min", SMA_S = 50, SMA_L = 200, units = 100)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0: # if we have final open position(s) (netting and hedging)
    api.close_all_for_symbol(trader.instrument)
    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
    time.sleep(20)
    print(api.get_closed_positions_summary()[col])
    trader.position = 0

In [ ]:
trader.data.tail(10)

In [ ]:
trader.data.tail(30).plot(figsize = (12, 8), secondary_y = "position")
plt.show()

__Strategy 2__: Bollinger Bands SMA 20 (minutes) / 1 Standard Deviation (200 ticks) 

## Stop here if you don´t want to see the solution!

###############################################################

In [ ]:
class BollTrader():
    
    def __init__(self, instrument, bar_length, SMA, dev, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        
        #*****************add strategy-specific attributes here******************
        self.SMA = SMA
        self.dev = dev
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["SMA"] = df[self.instrument].rolling(self.SMA).mean()
        df["Lower"] = df["SMA"] - df[self.instrument].rolling(self.SMA).std() * self.dev
        df["Upper"] = df["SMA"] + df[self.instrument].rolling(self.SMA).std() * self.dev
        df["distance"] = df[self.instrument] - df.SMA
        df["position"] = np.where(df[self.instrument] < df.Lower, 1, np.nan)
        df["position"] = np.where(df[self.instrument] > df.Upper, -1, df["position"])
        df["position"] = np.where(df.distance * df.distance.shift(1) < 0, 0, df["position"])
        df["position"] = df.position.ffill().fillna(0)
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0

    def report_trade(self, order, going):
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

In [ ]:
trader = BollTrader("EUR/USD", "1min", SMA = 20, dev = 1, units = 100)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0: # if we have final open position(s) (netting and hedging)
    api.close_all_for_symbol(trader.instrument)
    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
    time.sleep(20)
    print(api.get_closed_positions_summary()[col])
    trader.position = 0

In [ ]:
trader.data

In [ ]:
trader.data.tail(20)[["EUR/USD", "SMA", "Lower", "Upper"]].plot(figsize = (12, 8))
plt.show()

In [ ]:
api.close()

## Machine Learning Strategies (1) - Model Fitting

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [ ]:
data = pd.read_csv("five_minute.csv", parse_dates = ["time"], index_col = "time")

In [ ]:
data

In [ ]:
data["returns"] = np.log(data.div(data.shift(1)))

In [ ]:
data.dropna(inplace = True)

In [ ]:
data["direction"] = np.sign(data.returns)

In [ ]:
data

In [ ]:
lags = 2

In [ ]:
cols = []
for lag in range(1, lags + 1):
    col = "lag{}".format(lag)
    data[col] = data.returns.shift(lag)
    cols.append(col)
data.dropna(inplace = True)

In [ ]:
data

In [ ]:
lm = LogisticRegression(C = 1e6, max_iter = 100000, multi_class = "ovr")

In [ ]:
lm.fit(data[cols], data.direction)

In [ ]:
lm

__In-Sample "Prediction"__

In [ ]:
data["pred"] = lm.predict(data[cols])

In [ ]:
data

In [ ]:
hits = np.sign(data.direction * data.pred).value_counts()

In [ ]:
hits

In [ ]:
hit_ratio = hits[1.0] / sum(hits)
hit_ratio

In [ ]:
lm

__Saving the model__

In [ ]:
import pickle

In [ ]:
pickle.dump(lm, open("logreg.pkl", "wb"))

## Machine Learning Strategies (2) - Implementation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import fxcmpy
import time
from datetime import datetime
import pickle

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
class MLTrader():
    
    def __init__(self, instrument, bar_length, lags, model, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        
        #*****************add strategy-specific attributes here******************
        self.lags = lags
        self.model = model
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df = df.append(self.tick_data.iloc[-1]) # append latest tick (== open price of current bar)
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        cols = []
        for lag in range(1, self.lags + 1):
            col = "lag{}".format(lag)
            df[col] = df.returns.shift(lag)
            cols.append(col)
        df.dropna(inplace = True)
        df["position"] = lm.predict(df[cols])
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0

    def report_trade(self, order, going):
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

In [ ]:
lm = pickle.load(open("logreg.pkl", "rb"))

In [ ]:
lm

In [ ]:
trader = MLTrader("EUR/USD", "5min", lags = 2, model = lm, units = 100)

In [ ]:
trader.model

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

In [ ]:
api.unsubscribe_market_data(trader.instrument)
if len(api.get_open_positions()) != 0: # if we have final open position(s) (netting and hedging)
    api.close_all_for_symbol(trader.instrument)
    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
    time.sleep(20)
    print(api.get_closed_positions_summary()[col])
    trader.position = 0

In [ ]:
trader.data.tail(10)

In [ ]:
api.close()

##########################################################################################################

(you can find the corresponding Video for the following lines of code in __Section 23__: Cloud Deployment (AWS) | Scheduling Trading Sessions | Full Automation)

## How to Time/Schedule the End of a Trading Session (Simple Contrarian)

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
import time
from datetime import datetime

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
datetime.utcnow().time() # current utc time

In [ ]:
pd.to_datetime("17:30").time() # desired end of trading session (in utc time)

In [ ]:
if datetime.utcnow().time() >= pd.to_datetime("17:30").time():
    print("Stop the Trading Session!")

In [ ]:
class ConTrader():
    
    def __init__(self, instrument, bar_length, window, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************        
    
    def get_most_recent(self, period = "m1", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # define stop
        if recent_tick.time() >= pd.to_datetime("09:27").time():
            api.unsubscribe_market_data(self.instrument)
            if len(api.get_open_positions()) != 0:
                api.close_all_for_symbol(self.instrument)
                print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
                time.sleep(20)
                print(api.get_closed_positions_summary()[col])
                self.position = 0
                api.close()
        
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units * 2)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.create_market_buy_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            elif self.position == 1:
                order = api.create_market_sell_order(self.instrument, self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0

    def report_trade(self, order, going):  
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

In [ ]:
trader = ConTrader("EUR/USD", bar_length = "1min", window = 1, units = 100)

In [ ]:
trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

In [ ]:
api.close()